In [1]:
from preprocessing import pipeline as pipe

In [3]:
lemma = pipe.getTxtLemmatization(["train_pos.txt", "train_neg.txt"],
                        stopwords = False,
                        replace = True,
                        replace_stanford=False,
                        lemmatize = True,
                        outputfiles = ["redoing1_pos.txt","redoing1_neg.txt"])

Starting replacement
replacement done. 

Starting lemmatizing onreplaced_train_pos.txt
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
Starting lemmatizing onreplaced_train_neg.txt
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
Done 



In [17]:
from preprocessing import tokenizer as tok
voc = tok.build_vocab(frequency_treshold=10,
                file_name="redoing_vocab1.pkl",
                use_base_vocabulary=False,
                base_vocabulary_name="stanford_vocab.pkl",
                input_files= ["redoing1_pos.txt", "redoing1_neg.txt"])

Reading  redoing1_pos.txt
Reading  redoing1_neg.txt


In [18]:
from preprocessing import cooc

cooc_mat = cooc.build_cooc("redoing_vocab1.pkl",
               window_size=None,
               weighting="None",
               output_name="redoing_cooc1.pkl",
               input_files=["redoing1_pos.txt","redoing1_neg.txt"])

Working on  redoing1_pos.txt
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
Working on  redoing1_neg.txt
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
summing duplicates (this can take a while)


In [19]:
import embedding.pipeline as emb

emb1 = emb.get_glove_embedding(vocabulary_file="redoing_vocab1.pkl",
                        cooc_file="redoing_cooc1.pkl",
                        load_from_file=False,
                        file_name = "redoing_emb1.npz",
                        load_Stanford=True,
                        train=True,
                        save=True,
                        train_epochs=10,
                        train_eta=1e-3)              

In get glove embedding
Loading hyperparameters
Loading pre-trained Stanford embedding
Lost words:  0
Opening co-occurrence matrix
Started GloVe training
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9


In [10]:
from embedding import pipeline as pipe
import pickle
from embedding.embedding_base import EmbeddingBase
import pandas as pd
from embedding import sentence_embedding
import numpy as np
from classifier import Adaboost_classi as classy

In [20]:
with open("redoing_vocab1.pkl", "rb") as f:
    voc = pickle.load(f)
with open("redoing_cooc1.pkl", "rb") as f:
    cooc = pickle.load(f)
    
emb = EmbeddingBase(embedding_name = "redoing_emb1.npz",
                 embedding_dimension = 200,
                 vocabulary = voc,
                 cooc = cooc,
                 load = True)

mat1 = pipe.build_training_matrix(label = True,
                          embedding = emb,
                          input_files=["redoing1_pos.txt","redoing1_neg.txt"],
                          label_values=None,
                          input_entries=200000,
                          sentence_dimesion = 200,
                          output_location = "D:\\embedding_matrices\\redoing1_training_1")

mat2 = pipe.build_training_matrix(label = True,
                          embedding = emb,
                          input_files=["redoing1_pos.txt","redoing1_neg.txt"],
                          label_values=None,
                          input_entries=200000,
                          aggregation_fun=sentence_embedding.no_embeddings,
                          sentence_dimesion = 100,
                          output_location = "D:\\embedding_matrices\\redoing1_training_0")

redoing_emb1.npz
Working on  redoing1_pos.txt
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
Working on  redoing1_neg.txt
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
Number of lines read: 200000
Saving  D:\embedding_matrices\redoing1_training_1
Working on  redoing1_pos.txt
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
Working on  redoing1_neg.txt
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
Number of lines read: 200000
Saving  D:\embedding_matrices\redoing1_training_0


In [21]:
ada1 = classy.Adaboost_classi(embedding_dimension=200)
arr0 = np.load("D://embedding_matrices//redoing1_training_0.npz")
arr1 = np.load("D://embedding_matrices//redoing1_training_1.npz")
arr1 = arr1['arr_0']
arr0 = arr0['arr_0']
x = arr0[:,:-1]
y = arr0[:,-1]
ada1.build()
ada1.train(x,y)
x = arr1[:,:-1]
y = arr1[:,-1]
ada1.build()
ada1.train(x,y)

Building model.
Training model
[0.676425 0.66025  0.65     0.6452   0.64475 ]
Building model.
Training model
[0.744975 0.74595  0.734775 0.7372   0.732075]
